# 1. Get Data

In [1]:
import os
import requests
import zipfile
from pathlib import Path
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...") 
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

data\pizza_steak_sushi directory exists.
Unzipping pizza, steak, sushi data...


In [2]:
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [3]:
data_transform = transforms.Compose([ 
    # Resize image
    transforms.Resize(size=(64, 64)),
    # Flip the image randomly, horizontal
    transforms.RandomHorizontalFlip(p=0.5), # 50% of all images will be flip horisontaly
    # Turn image in to Tensor
    transforms.ToTensor()
])

# 2. Create Datasets and DataLoaders (data_setup.py)

In [4]:
# import os
# os.mkdir("going_modular")

In [5]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for 
image classification data.
"""
import os 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloader(
    train_dir: str,
    test_dir: str,
    batch_size: int,
    transforms: transforms.Compose,
    num_workers: int=NUM_WORKERS):
    """Creates training and testing DataLoaders.

      Takes in a training directory and testing directory path and turns
      them into PyTorch Datasets and then into PyTorch DataLoaders.
    
      Args:
        train_dir: Path to training directory.
        test_dir: Path to testing directory.
        transform: torchvision transforms to perform on training and testing data.
        batch_size: Number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader.
    
      Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
        Example usage:
          train_dataloader, test_dataloader, class_names = \
            = create_dataloaders(train_dir=path/to/train_dir,
                                 test_dir=path/to/test_dir,
                                 transform=some_transform,
                                 batch_size=32,
                                 num_workers=4)
      """
    train_dataset = datasets.ImageFolder(root=train_dir, transform=transforms)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=transforms)
    
    class_names = train_dataset.classes
    
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=NUM_WORKERS, shuffle=True, pin_memory=True)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=NUM_WORKERS, pin_memory=True)

    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


In [6]:
from going_modular import data_setup
train_dataloader, test_dataloader, class_names = data_setup.create_dataloader(train_dir=train_dir,
                                                                              test_dir=test_dir,
                                                                              batch_size=32,
                                                                              transforms=data_transform)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x12bb7ba5940>,
 ['pizza', 'steak', 'sushi'])

In [7]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch 
from torch import nn

class TinyVGG(nn.Module):
    def __init__(self, 
                 input_shape: int,
                 hidde_layers: int,
                 output_shape: int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidde_layers, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidde_layers, out_channels=hidde_layers, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidde_layers, out_channels=hidde_layers, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidde_layers, out_channels=hidde_layers, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.clasiffier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidde_layers*13*13, # RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x1690 and 10x3), Output shepe of conv_block_2: torch.Size([32, 10, 13, 13]) 10*13*13=1690 in this case our hiddin_layers = 10, so we need to * hidden_layers on 13*13.
                     out_features=output_shape)
        )
        
    def forward(self, x):
       # x = self.conv_block_1(x)
       #print(f"Output shepe of conv_block_1: {x.shape}")
       # x = self.conv_block_2(x)
       #print(f"Output shepe of conv_block_2: {x.shape}")
       # x = self.clasiffier(x)
       #print(f"Output shepe of clasiffier: {x.shape}")
       #return x
       return self.clasiffier(self.conv_block_2(self.conv_block_1(x))) # better and faster option than just return  x 

Overwriting going_modular/model_builder.py


In [8]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch
from torch import nn
from typing import Dict, List, Tuple

device = "cuda" if torch.cuda.is_available() else "cpu"

def train_step(model: nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: nn.Module,
               optimizer: torch.optim.Optimizer):
    
    train_loss, train_acc = 0, 0
    model.train()

    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)

        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    
    return train_loss, train_acc

def test_step(model: nn.Module,
          data_loader: torch.utils.data.DataLoader,
          loss_fn: nn.Module):

    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for batch, (X, y) in enumerate(data_loader):
            X, y = X.to(device), y.to(device)
            test_pred = model(X)
    
            loss = loss_fn(test_pred, y)
            test_loss += loss.item()
            test_pred_labels = test_pred.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))
    
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    
    return test_loss, test_acc

def train(model: nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          device = device):
    # Create empty dictionary for results.
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []}

    for epoch in range(epochs):
        train_loss, train_acc = train_step(model=model,
                                           data_loader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model, 
                                        data_loader=test_dataloader,
                                        loss_fn=loss_fn)
        print(f"Eprch: {epoch} | Train Loss: {train_loss:.2f} | Train Acc: {train_acc:.1f} % | Test Loss: {test_loss:.2f} | Test Acc: {test_acc:.1f} % ")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

Overwriting going_modular/engine.py


In [9]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.
  
  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)
  
  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Overwriting going_modular/utils.py


In [10]:
%%writefile going_modular/train.py
"""
Trains a PyTorch imageclassification model
"""

import os
import torch
from torch import nn
from timeit import default_timer as timer
from torchvision import datasets, transforms
import data_setup, engine, model_builder, utils

NUM_EPOCHS = 5
NUM_BATCHS =32 
HIDDEN_UNITS = 10
LEARNIG_RATE = 0.001
NUM_WORKERS = os.cpu_count()

# Setup hypperparameters 
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

device = "cuda" if torch.cuda.is_available() else "cup"

data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
# Create DataLoader, Class_names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloader(train_dir=train_dir,
                                                                              test_dir=test_dir,
                                                                              batch_size=NUM_BATCHS,
                                                                              transforms=data_transform)

#Create Model
model = model_builder.TinyVGG(input_shape=3,
                              hidde_layers=10,
                              output_shape=len(class_names))

#Setup Loss and Optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNIG_RATE)

# Strat training
strat_time = timer()
engine.train(model=model, 
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)
end_time = timer()
print(f"Total time: {end_time - strat_time:.1f} seconds.")

#Save Model
utils.save_model(model=model,
                 target_dir="models",
                 model_name="moduler_script_TinyVGG.pt")

Overwriting going_modular/train.py


In [ ]:
!python going_modular/train.py